# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    DD-Mmm-YYYY**

**News Source / Catalyst:**

**Markets (s) Impacted:** (FX, EQ, COM, FI, EM, etc.)

**Trade Taxonomy:** (RV Macro / RV Sector / Structural FX / Event-Driven / Thematic SS)

**Trade Idea (1 sentence):**
> 

**Client/Desk Pitch (30 seconds):**
>

**Why Now:**
- 
- 
- 

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- 
- 
- 

**Long Leg (Tickers + Rationale):**
- 

**Short Leg (Tickers + Rationale):**
- 

### **Weights:** equal-weight / vol-weight / beta-neutral

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |                  |       |
| Oil Beta             |                  |       |
| Rates Duration       |                  |       |
| Growth vs Value      |                  |       |
| Risk-On/Off          |                  |       |
| China/EM Beta        |                  |       |
| Volatility Regime    |                  |       |

**Catalyst Type:** (policy / earnings / data / supply-demand / regime shift)  
**Realisation Window:** (days/weeks/quarters)  
**Holding Period Selected:**  
**Alignment Score:** Good / Partial / Poor

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** (units per leg)  
**Stop-Loss:** (portfolio-level %)  
**Take-Profit (optional):**  
**Re-entry:** Yes/No  
**FX-Hedging Required:** Yes/No  
**Liquidity Notes:** ADV / spreads / ETF AUM  

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [ ] Information

---


# **PYTHON IMPLEMENTATION**

In [18]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [19]:
# --- Parameters ---

long_tickers = ["EWP", "EWI", "PGAL", "GREK"]     # EDIT HERE
short_tickers = ["EWG", "EWQ"]                    # EDIT HERE

start_date = "2020-01-01"                         # Backtest window
holding_period = 10                               # days
stop_loss = -0.02                                 # -2% stop
take_profit = None                                # optional e.g. +0.03 for +3%

# --- Weightings ---

weights_long = 1 / len(long_tickers)
weights_short = 1 / len(short_tickers)


In [20]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_27923/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  6 of 6 completed


In [21]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [22]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
146,2025-10-24,0.0375
147,2025-11-07,0.0140
148,2025-11-21,-0.0109
149,2025-12-08,0.0069
150,2025-12-22,-0.0039


In [23]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         151.0000
Win Rate               0.4967
Average PnL            0.0008
Best Trade             0.0483
Worst Trade           -0.0333
Stop-Out Frequency     0.1325
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 
- Win Rate:
- Avg PnL:
- Best:
- Worst: 
- Stop-Outs: 

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [24]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [25]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [26]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
146,RV-AF923EA9,2026-01-11 18:16:15,Macro RV Spread,RV Macro Spread,"EWP, EWI, PGAL, GREK","EWG, EWQ",2025-10-24,2025-11-06,-0.2591,1.3385,3.7502,-2.0000,None,CLOSED,13,
147,RV-A831A5DD,2026-01-11 18:16:15,Macro RV Spread,RV Macro Spread,"EWP, EWI, PGAL, GREK","EWG, EWQ",2025-11-07,2025-11-20,1.1615,1.8258,1.4001,-2.0000,None,CLOSED,13,
148,RV-9AA093E7,2026-01-11 18:16:15,Macro RV Spread,RV Macro Spread,"EWP, EWI, PGAL, GREK","EWG, EWQ",2025-11-21,2025-12-05,1.9345,1.9139,-1.0886,-2.0000,None,CLOSED,14,
149,RV-C73837BC,2026-01-11 18:16:15,Macro RV Spread,RV Macro Spread,"EWP, EWI, PGAL, GREK","EWG, EWQ",2025-12-08,2025-12-19,2.0662,2.4549,0.6917,-2.0000,None,CLOSED,11,
150,RV-BADEBA6E,2026-01-11 18:16:15,Macro RV Spread,RV Macro Spread,"EWP, EWI, PGAL, GREK","EWG, EWQ",2025-12-22,2026-01-06,2.6415,2.5213,-0.3884,-2.0000,None,CLOSED,15,


In [27]:
blotter.to_csv("trade_blotter.csv", index=False)